In [35]:
!pip install torch
!pip install gluonts
!pip install huggingface_hub
!pip install matplotlib pandas numpy

In [36]:
!pip install git+https://github.com/time-series-foundation-models/lag-llama.git
!pip install lag-llama


  Cloning https://github.com/time-series-foundation-models/lag-llama.git to /private/var/folders/ls/yshfj6q93s9_16nz44mv2pb00000gn/T/pip-req-build-kouvdd7q
  Running command git clone --filter=blob:none --quiet https://github.com/time-series-foundation-models/lag-llama.git /private/var/folders/ls/yshfj6q93s9_16nz44mv2pb00000gn/T/pip-req-build-kouvdd7q
  Resolved https://github.com/time-series-foundation-models/lag-llama.git to commit df7531a83a19b3c6a0222d703ca9bf59ef7a6ab9
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [37]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split
from lag_llama.gluon.estimator import LagLlamaEstimator

# Configuration
PREDICTION_LENGTH = 12
CONTEXT_LENGTH = 64
BATCH_SIZE = 4
TEST_LENGTH = 24

In [38]:
# Generate synthetic data
def generate_time_series_data(periods=200, n_variables=3, univariate=False):
    index = pd.date_range(start="2021-01-01", periods=periods, freq="H")
    if univariate:
        data = np.random.randn(periods) + np.sin(np.arange(periods) * 0.1)
        return pd.DataFrame(data, index=index, columns=["target"])
    else:
        columns = [f"series_{i}" for i in range(n_variables)]
        data = np.random.randn(periods, n_variables)
        return pd.DataFrame(data, index=index, columns=columns)

df_univariate = generate_time_series_data(univariate=True)
df_multivariate = generate_time_series_data()

# Convert to GluonTS format
gluon_uni = PandasDataset(dict(df_univariate))
gluon_multi = PandasDataset(dict(df_multivariate))

# Split data
train_uni, test_template_uni = split(gluon_uni, offset=-TEST_LENGTH)
train_multi, test_template_multi = split(gluon_multi, offset=-TEST_LENGTH)

test_data_uni = test_template_uni.generate_instances(
    prediction_length=PREDICTION_LENGTH,
    windows=TEST_LENGTH // PREDICTION_LENGTH,
    distance=PREDICTION_LENGTH
)

test_data_multi = test_template_multi.generate_instances(
    prediction_length=PREDICTION_LENGTH,
    windows=TEST_LENGTH // PREDICTION_LENGTH,
    distance=PREDICTION_LENGTH
)

# Create LagLlama model (with corrected frequency codes)
lagllama_estimator = LagLlamaEstimator(
    prediction_length=PREDICTION_LENGTH,
    context_length=CONTEXT_LENGTH,
    lr=1e-3,
    batch_size=BATCH_SIZE,
    lags_seq=[],  # Empty lags to avoid frequency issues
    trainer_kwargs={"max_epochs": 3, "logger": False, "enable_progress_bar": True}
)



/var/folders/ls/yshfj6q93s9_16nz44mv2pb00000gn/T/ipykernel_29132/1985302375.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  index = pd.date_range(start="2021-01-01", periods=periods, freq="H")


In [ ]:
# Train and predict - Univariate
print("Training LagLlama on univariate data...")
predictor_uni = lagllama_estimator.train(train_uni)
forecasts_uni = predictor_uni.predict(test_data_uni.input)

# Extract results - Univariate
results_uni = {}
for col in df_univariate.columns:
    results_uni[col] = []

for forecast in forecasts_uni:
    series_name = forecast.item_id
    if hasattr(forecast, 'samples'):
        median_pred = np.median(forecast.samples, axis=0)
    else:
        median_pred = forecast.mean
    results_uni[series_name].extend(median_pred.tolist())

print("Univariate Results:")
print(results_uni)


Training LagLlama on univariate data...


ValueError: max() arg is an empty sequence